<a href="https://colab.research.google.com/github/KhondamirRustamov/bioinformatics301/blob/main/lab_docking/docking_vina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab для использования AutoDock Vina**

Супер простой вариант использования докинга

*Хондамир Рустамов*
©



In [ ]:
#@title Загрузка необходимых библиотек для работы с AutoDock Vina

#Installing py3Dmol using pip
!pip -q install py3Dmol
#Installing biopython using pip
!pip -q install biopython
#Installing pdb2pqr v3.0 using pip
!pip -q install pdb2pqr
#We will also install kora for using RDkit
!pip -q install kora
!pip -q install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 38.3 MB/s eta 0:00:00


In [ ]:
#@title Проверка правильности работы загруженных библиотек (можно пропустить данный шаг)
#Importing py3Dmol for safety
import py3Dmol
#Checking that pdb2pqr was properly installed
!pdb2pqr30 -h

In [ ]:
#@title Загрузка CondaColab и сторонних программ для подготовки белка-рецептора и лиганда 
#@markdown (на данном этапе колаб может остановиться, не обращайте на это внимание и продолжайте выполнение следующего куска кода)

#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

In [ ]:
#@title Загрузка самой программы AutoDock Vina

#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary
%%bash
wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
tar xzvf autodock_vina_1_1_2_linux_x86.tgz

In [ ]:
#@title Добавление AutoDock Vina в 'PATH'
%alias vina /content/autodock_vina_1_1_2_linux_x86/bin/vina
%alias vina_split /content/autodock_vina_1_1_2_linux_x86/bin/vina_split

In [ ]:
#@title Подготовка рецептора
#@markdown Введите название файла с .pdb белка-рецептора (только название файла без .pdb)
receptor = '5ybf' #@param {type: 'string'}
#First, using pdb2pqr to parameterize our receptor with AMBER99ff, maintaining
#the chain IDs and setting up the receptor at a pH of 7.4
!pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 {receptor}.pdb {receptor}.pqr

#Then, convert the .pqr file into a .pdbqt file while deleting non-polar
#hydrogens but without changing the AMBER parameters added to the protein
!prepare_receptor4.py -r {receptor}.pqr -o {receptor}.pdbqt -C -U nphs_lps -v

In [ ]:
#@title Подготовка лиганда по файлу Smiles

smiles = 'C(CC(=O)[O-])C(C(=O)[O-])[NH3+]' #@param {type: 'string'}
ligand = 'ligand'
file = open('ligand.smiles', 'w')
file.write(smiles)
file.close()

!obabel ligand.smiles -O ligand.mol2 --gen3d best -p 7.4 --canonical
#Parameterizing and adding Gasteiger charges into our protein using MGLtools
#Adding -z leads to a rigid ligand without any torsions
!prepare_ligand4.py -l ligand.mol2 -o ligand.pdbqt -U nphs_lps -v
#NOTE: for some reason, MGLtools does not recognize the ligand when inside a different folder
#Here we are deleting the temporary PDB file required for generating the PDBQT file

In [ ]:
#@title Подготовка лиганда по файлy .mol2 из Charmm-GUI
#@markdown Введите название файла .mol2 с лигандом (просто название без .mol2)
ligand = '' #@param {type: 'string'}
!prepare_ligand4.py -l {ligand}.mol2 -o {ligand}.pdbqt -U nphs_lps -v

In [ ]:
#@title Запуск функций для визуализации белка-рецептора
#These definitions will enable loading our protein and then
#drawing a box with a given size and centroid on the cartesian space
#This box will enable us to set up the system coordinates for the simulation
#
#HINT: The active site of the HIV-2 protease is near the beta strands in green
#
#ACKNOWLEDGE: This script is largely based on the one created by Jose Manuel 
#Napoles Duarte, Physics Teacher at the Chemical Sciences Faculty of the 
#Autonomous University of Chihuahua (https://github.com/napoles-uach)
#
#First, we define the grid box
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

#Next, we define how the protein will be shown in py3Dmol
#Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile,resids):
  mol1 = open(prot_PDBfile, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})

#Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,600)
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.zoomTo()
  mol_view.show() 

В данной ячейке нам нужно определить:


*   Номера подвижных аминокислот в рецепторе (AutoDock Vina проводит flexible docking, то есть при докинге лиганда в рецептор радикалы выбранных  аминокислот рецептора также изменяют свое положение)
*   Расположение и размеры бокса для докинга (в какой части рецептора проводить докинг)



In [ ]:
#@title Визуализации белка-рецептора и определение центра и размеров бокса для докинга 

import py3Dmol

flexible_resids='142, 143, 193, 191, 220, 89, 91, 96' #@param {type: 'string'}
flexible_resids_list = flexible_resids.split(',')

x_center = '0' #@param {type: 'string'}
y_center = '-3' #@param {type: 'string'}
z_center = '2' #@param {type: 'string'}

#@markdown Вставьте размеры бокса докинга по осям x, y, z
x_size = '16' #@param {type: 'string'}
y_size = '16' #@param {type: 'string'}
z_size = '16' #@param {type: 'string'}

viewprotgrid(f'{receptor}.pdb', 
             flexible_resids_list, 
             bxi=int(x_center), 
             byi=int(y_center), 
             bzi=int(z_center),
             bxf=int(x_size),
             byf=int(y_size),
             bzf=int(z_size),)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Подготовка файла настроек для Докинга (config)

with open("config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write(f"receptor = {receptor}.pdbqt \n")
  f.write(f"ligand = {ligand}.pdbqt \n")
  f.write(f"#flex = [{flexible_resids}] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES  
  f.write(f"center_x = {x_center} \n")
  f.write(f"center_y = {y_center} \n")
  f.write(f"center_z = {z_center} \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write(f"size_x = {x_size} \n")
  f.write(f"size_y = {y_size} \n")
  f.write(f"size_z = {z_size} \n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

In [ ]:
#@title Запуск докинга на AutoDock Vina
#Changing directory to the single docking folder

#Executing AutoDock Vina with our configuration file
%vina --config config_singledock --out output.pdbqt --log log.txt
#Exiting the execution directory


/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 2 CPUs
Reading input ... done.
Set

In [ ]:
#@title Визуализация лучшего результата

!obabel -ipdbqt output.pdbqt -opdb -O output.pdb -m
view=py3Dmol.view()
view.setBackgroundColor('white')
#Loading the target protein as PDB file
view.addModel(open('5ybf.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
#view.addStyle({'stick':{'colorscheme':'greenCarbon'}})
view.zoomTo()
#Loading the docking pose
view.addModel(open('output1.pdb', 'r').read(),'pdb')
view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
view.show()

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
9 molecules converted
9 files output. The first is output1.pdb


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol